In [32]:
import andes
import pandapower as pp
from jams import dcopf

from andes.interop.pandapower import build_group_table, add_gencost, make_GSF

import numpy as np
import pandas as pd

In [33]:
print(andes.__version__)
print(pp.__version__)

1.6.5.post32.dev0+gafd82414
2.8.0


No line limits:
Only linear cost or only quadratic cost, the results are the same.
With line limits:
not the same with pandapower or matpower.

In [34]:
def bc_dc(case_path, case_name):
    case = case_path + case_name
    ssa = andes.load(case,
                    setup=True,
                    no_output=True,
                    default_config=False)
    build_group_table(ssa, 'StaticGen', ['u', 'idx', 'p0'])

    ssd = dcopf()
    ssd.from_andes(ssa)
    ssd.cost.c1 = range(1,ssd.gen.shape[0]+1)
    ssd.cost.c2 = 0
    ssd.build()
    ssd.mdl.optimize()

    from andes.interop.pandapower import to_pandapower
    ssp = to_pandapower(ssa)

    # add gen cost
    gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
    gen_cost[:, 4] = ssd.cost.c2  # c2
    gen_cost[:, 5] = ssd.cost.c1  # c1

    add_gencost(ssp, gen_cost)
    pp.rundcopp(ssp)
    res_pp = ssp.res_gen
    res_jm = ssd.get_res()
    res_comp = pd.concat([res_pp, res_jm], axis=1)
    res_comp.p_sch = res_comp.p_sch * ssd.mva
    return res_comp.round(4)

case_path = '/Users/jinningwang/Documents/work/andes/jwang/jams/cases/'
case5 = 'pjm5bus.xlsx'
case14 = 'case14_lim.xlsx'
case39 = 'case39.xlsx'
res5 = bc_dc(case_path, case5)
res14 = bc_dc(case_path, case14)
res39 = bc_dc(case_path, case39)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 4 columns and 46 nonzeros
Model fingerprint: 0x8e417ab2
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+01, 2e+01]
Presolve removed 15 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.000000000e+00
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 35 rows, 5 columns and 135 nonzeros
Model fingerprint: 0x0a0

gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 10 columns and 492 nonzeros
Model fingerprint: 0x5c2ed624
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [5e+00, 1e+01]
  RHS range        [8e-01, 6e+01]
Presolve removed 52 rows and 0 columns
Presolve time: 0.00s
Presolved: 19 rows, 16 columns, 171 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.2542300e+01   1.375001e+02   0.000000e+00      0s
       8    3.1019679e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.101967924e+02


gen vm_pu > bus max_vm_pu for gens [5]. Setting bus limit for these gens.


In [35]:
res5

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,1300.0,46.9205,5.6420,1.0,0,1300.0
1,-100.0,171.0009,-1.3438,1.0,2,-100.0
2,-100.0,-39.4140,4.3401,1.0,4,-100.0
3,-100.0,181.1884,0.0000,1.0,3,-100.0


In [36]:
res14

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,140.0,43.5571,0.7651,1.0,2,140.0
1,100.0,25.0753,-1.1962,1.0,3,100.0
2,19.0,12.7309,-6.6211,1.0,4,19.0
3,0.0,17.6235,-6.4877,1.0,5,0.0
4,0.0,-16.5493,0.0000,1.0,1,0.0


In [37]:
res39

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,1040.00,161.7616,24.2665,1.0,1,1040.0000
1,725.00,206.9649,13.3613,1.0,3,725.0000
2,652.00,108.2928,19.3832,1.0,4,652.0000
3,508.00,166.6884,17.5519,1.0,5,508.0000
4,687.00,210.6614,22.1292,1.0,6,687.0000
5,580.00,100.1648,25.1579,1.0,7,580.0000
6,564.00,-1.3694,21.9255,1.0,8,363.9871
7,865.00,21.7327,27.3544,1.0,9,808.2620
8,633.23,78.4674,-4.7612,1.0,10,287.1353
9,0.00,221.5745,0.0000,1.0,2,602.8455


In [38]:
case_path = '/Users/jinningwang/Documents/work/andes/jwang/jams/cases/'
case14 = 'case39_lim.xlsx'

case = case_path + case14
ssa = andes.load(case,
                setup=True,
                no_output=True,
                default_config=False)
build_group_table(ssa, 'StaticGen', ['u', 'idx', 'p0'])

ssd = dcopf()
ssd.from_andes(ssa)
ssd.cost.c1 = range(1,ssd.gen.shape[0]+1)
ssd.cost.c2 = 0
ssd.build()
ssd.mdl.optimize()

from andes.interop.pandapower import to_pandapower
ssp = to_pandapower(ssa)

# add gen cost
gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
gen_cost[:, 4] = ssd.cost.c2  # c2
gen_cost[:, 5] = ssd.cost.c1  # c1

add_gencost(ssp, gen_cost)
pp.rundcopp(ssp)
res_pp = ssp.res_gen
res_jm = ssd.get_res()
res_comp = pd.concat([res_pp, res_jm], axis=1)
res_comp.p_sch = res_comp.p_sch * ssd.mva

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 10 columns and 492 nonzeros
Model fingerprint: 0x5c2ed624
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [5e+00, 1e+01]
  RHS range        [8e-01, 6e+01]
Presolve removed 52 rows and 0 columns
Presolve time: 0.00s
Presolved: 19 rows, 16 columns, 171 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.2542300e+01   1.375001e+02   0.000000e+00      0s
       8    3.1019679e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.101967924e+02


gen vm_pu > bus max_vm_pu for gens [5]. Setting bus limit for these gens.


ssd.gen

In [39]:
ssd.cost

,idx,c2,c1,c0,cr
0,1,0,1,0,0
1,3,0,2,0,0
2,4,0,3,0,0
3,5,0,4,0,0
4,6,0,5,0,0
5,7,0,6,0,0
6,8,0,7,0,0
7,9,0,8,0,0
8,10,0,9,0,0
9,2,0,10,0,0


In [40]:
ssd.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,ctrl,ramp_agc,ramp_10,ramp_30
0,1,1.0,PV 30,100.0,345.0,30,2.5,10.4,0.0,1.0499,1,100,100,100
1,3,1.0,PV 32,100.0,345.0,32,6.5,7.25,0.0,0.9841,1,100,100,100
2,4,1.0,PV 33,100.0,345.0,33,6.32,6.52,0.0,0.9972,1,100,100,100
3,5,1.0,PV 34,100.0,345.0,34,5.08,5.08,0.0,1.0123,1,100,100,100
4,6,1.0,PV 35,100.0,345.0,35,6.5,6.87,0.0,1.0494,1,100,100,100
5,7,1.0,PV 36,100.0,345.0,36,5.6,5.8,0.0,1.0636,1,100,100,100
6,8,1.0,PV 37,100.0,345.0,37,5.4,5.64,0.0,1.0275,1,100,100,100
7,9,1.0,PV 38,100.0,345.0,38,8.3,8.65,0.0,1.0265,1,100,100,100
8,10,1.0,PV 39,100.0,345.0,39,10.0,11.0,0.0,1.03,1,100,100,100
9,2,1.0,Slack 31,100.0,345.0,31,6.77871,6.46,0.0,0.982,1,100,100,100


In [41]:
res_comp.round(4)

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,1040.00,161.7616,24.2665,1.0,1,1040.0000
1,725.00,206.9649,13.3613,1.0,3,725.0000
2,652.00,108.2928,19.3832,1.0,4,652.0000
3,508.00,166.6884,17.5519,1.0,5,508.0000
4,687.00,210.6614,22.1292,1.0,6,687.0000
5,580.00,100.1648,25.1579,1.0,7,580.0000
6,564.00,-1.3694,21.9255,1.0,8,363.9871
7,865.00,21.7327,27.3544,1.0,9,808.2620
8,633.23,78.4674,-4.7612,1.0,10,287.1353
9,0.00,221.5745,0.0000,1.0,2,602.8455


In [47]:
ssd.bus

,idx,u,name,Vn,vmax,vmin,v0,a0,area,zone,owner
uid,,,,,,,,,,,
0,1,1.0,Bus 1,345.0,1.06,0.94,1.039384,-0.236258,2,1,None
1,2,1.0,Bus 2,345.0,1.06,0.94,1.048494,-0.170785,2,1,None
2,3,1.0,Bus 3,345.0,1.06,0.94,1.030708,-0.214263,2,1,None
3,4,1.0,Bus 4,345.0,1.06,0.94,1.004460,-0.220378,1,1,None
4,5,1.0,Bus 5,345.0,1.06,0.94,1.006006,-0.195343,1,1,None
5,6,1.0,Bus 6,345.0,1.06,0.94,1.008226,-0.181660,1,1,None
6,7,1.0,Bus 7,345.0,1.06,0.94,0.998397,-0.222628,1,1,None
7,8,1.0,Bus 8,345.0,1.06,0.94,0.997872,-0.232754,1,1,None
8,9,1.0,Bus 9,345.0,1.06,0.94,1.038332,-0.247460,1,1,None


In [56]:
net = pd.DataFrame()
net['bus'] = ssd.bus[['idx']]
net = pd.merge(left=net, right=ssd.load[['bus', 'p0']].rename(columns={'p0': 'load'}), on='bus', how='left')
net = pd.merge(left=net, right=ssd.gen[['bus', 'p0']].rename(columns={'p0': 'gen'}), on='bus', how='left')
net.fillna(0, inplace=True)
net['net'] = net['gen'] - net['load']
net

gsf = make_GSF(ssp)
lf = np.matmul(gsf, net['net'])
lf.shape

(35,)

In [65]:
net.sum()

bus     780.00000
load     62.54230
gen      62.97871
net       0.43641
dtype: float64

In [66]:
ssd.gen_gsf

,idx,name,bus,0,1,2,3,4,5,6,...,25,26,27,28,29,30,31,32,33,34
0,1,PV 30,30,-0.21065,0.21065,0.63149,0.157859,0.539748,0.091742,0.504101,...,-0.157859,0.0,0.0,-0.0,0.0,0.157859,0.157859,-0.0,-0.0,-0.0
9,2,Slack 31,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,PV 32,32,-0.023646,0.023646,-0.012532,-0.011114,0.016351,-0.028883,0.215511,...,0.011114,-0.0,-0.0,0.0,0.0,-0.011114,-0.011114,-0.0,-0.0,-0.0
2,4,PV 33,33,-0.108363,0.108363,-0.003899,-0.104464,0.347569,-0.351468,0.510131,...,0.104464,-0.0,-0.0,0.0,0.0,-0.104464,-0.104464,-0.0,-0.0,-0.0
3,5,PV 34,34,-0.108363,0.108363,-0.003899,-0.104464,0.347569,-0.351468,0.510131,...,0.104464,-0.0,-0.0,0.0,0.0,-0.104464,-0.104464,-0.0,-0.0,-0.0
4,6,PV 35,35,-0.108363,0.108363,-0.003899,-0.104464,0.347569,-0.351468,0.510131,...,0.104464,-0.647436,0.352564,0.352564,0.0,-0.104464,-0.104464,-0.0,-0.0,-0.0
5,7,PV 36,36,-0.108363,0.108363,-0.003899,-0.104464,0.347569,-0.351468,0.510131,...,0.104464,-0.524359,-0.524359,0.475641,-1.0,-0.104464,-0.104464,-0.0,-0.0,-0.0
6,8,PV 37,37,-0.20052,0.20052,0.557314,-0.757834,0.52762,0.029694,0.506342,...,-0.242166,0.0,0.0,-0.0,0.0,0.242166,0.242166,-0.0,-0.0,-0.0
7,9,PV 38,38,-0.162472,0.162472,0.278724,-0.441196,0.48207,-0.203346,0.514759,...,-0.558804,-0.0,-0.0,0.0,0.0,-0.441196,0.558804,-0.5,-0.5,-0.5
8,10,PV 39,39,0.39831,-0.39831,0.318313,0.079997,0.270626,0.047687,0.242074,...,-0.079997,0.0,0.0,-0.0,0.0,0.079997,0.079997,0.0,0.0,0.0


In [63]:
lineres = ssd.line[['idx', 'rate_a']]
lineres['lf'] = lf
lineres

,idx,rate_a,lf
0,Line_1,6.0,-1.783537
1,Line_2,10.0,0.807537
2,Line_3,5.0,3.334301
3,Line_4,5.0,-2.617838
4,Line_6,5.0,0.541154
5,Line_7,5.0,-0.426853
6,Line_8,6.0,-1.776858
7,Line_9,5.0,-2.681988
8,Line_10,12.0,-5.147537
9,Line_11,9.0,3.370680
